In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' # Must be set before importing TF to supress messages
os.environ["CUDA_VISIBLE_DEVICES"]= '3'

import tensorflow as tf
import numpy as np
from utils.configs import config
from typing import List
import pymf
import sklearn
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

def load_VGG_model(img_height: int, img_width: int, lr: int, loss: tf.keras.losses.Loss, metrics: List[str], trainable: True) -> tf.keras.Model:
    """ Loads VGG-16 model.

    Args:
        img_height (int): Image height.
        img_width (int): Image width.
        lr (int): Learning rate.
        loss (tf.keras.losses.Loss): Model loss.
        metrics (List[str]): Training metrics.
        trainable (True): Set if model weights should be kept frozen or not.

    Returns:
        tf.keras.Model: TensorFlow VGG-16 model.
    """
    model = tf.keras.applications.vgg16.VGG16(input_shape=(img_height, img_width, 3))
    model.trainable = trainable
    model.compile(optimizer=tf.keras.optimizers.Adam(lr, epsilon=0.1),
                loss=loss,
                metrics=metrics)

    return model

In [9]:
def retrieve_pcs(data, n_comp):
    svd = np.linalg.svd(data, full_matrices=True) # Take full matrix - P is already ready
    U = svd[0]
    S = np.vstack([np.diag(svd[1]), np.zeros((svd[0].shape[0]-min(data.shape), svd[1].shape[0]))]) # Reconstruct true S matrix
    VT = svd[2]

    # Extract the top n_comp principal components
    U_f = U[:, :n_comp]
    S_f = S[:n_comp, :n_comp]
    V_f = VT[:n_comp, :] # rows of VT contain the principle axes

    return U_f, S_f, V_f

In [3]:
model = load_VGG_model(img_height=224, img_width=224, lr=0.001, loss=tf.keras.losses.CategoricalCrossentropy(), metrics=['accuracy'], trainable=True)

In [4]:
kernel = model.get_layer('block4_conv2').kernel
flat_kernel = tf.reshape(kernel, [-1, kernel.shape[-1]]).numpy()

In [ ]:
normalised_fk = ((flat_kernel - np.mean(flat_kernel, axis=0)) / np.std(flat_kernel, axis=0))

SNMF

In [ ]:
nmf = pymf.SNMF(flat_kernel.T, num_bases=90)
nmf.factorize(niter=100)

In [ ]:
print(f'Original Data Shape: {flat_kernel.shape}')
print(f'W Shape: {nmf.W.shape}, H shape: {nmf.H.shape}')
print(nmf.H)

Original Data Shape: (4608, 512)
W Shape: (512, 90), H shape: (90, 4608)
[[0.73764775 0.7528971  0.59190911 ... 0.958576   0.6760751  0.76307495]
 [0.30181689 0.25709903 0.84324327 ... 0.24214831 0.00519485 0.01294836]
 [0.1366661  0.70271647 0.26033878 ... 0.57364907 0.47377842 0.05059574]
 ...
 [0.76121811 0.76182188 0.59391574 ... 0.43254617 0.35416131 0.64657014]
 [0.73507097 0.85946173 0.01798513 ... 0.30071964 0.63542806 0.7460896 ]
 [0.60270452 0.86541689 0.59534312 ... 0.74906893 0.3799919  0.92967351]]


PCA

In [10]:
u,s,v = retrieve_pcs(flat_kernel.T, 90)

In [13]:
u.shape

(512, 90)

In [48]:
pca = PCA(90)
reduced = pca.fit_transform(flat_kernel)
X_re_orig = pca.inverse_transform(reduced)

Performance Metrics

In [86]:
rss_nmf = np.sum(np.square(flat_kernel - nmf.W@nmf.H))
print(f'SNMF: Residual Sum Square Error: {rss_nmf}')

rss_svd = np.sum(np.square(flat_kernel - u@s@v))
print(f'Manual SVD PCA: Residual Sum Square Error: {rss_svd}')

rss_pca = np.sum(np.square(flat_kernel - X_re_orig.T))
print(f'Sklearn PCA: Residual Sum Square Error: {rss_pca}')

SNMF: Residual Sum Square Error: 55.002846404742044
Manual SVD PCA: Residual Sum Square Error: 53.68510261014783
Sklearn PCA: Residual Sum Square Error: 53.781124114990234


In [80]:
snmf_var = sklearn.metrics.explained_variance_score(flat_kernel, nmf.W@nmf.H)
print(f'SNMF: Explained Variance: {snmf_var}')

svd_var = sklearn.metrics.explained_variance_score(flat_kernel, u@s@v)
print(f'Manual SVD PCA: Explained Variance: {svd_var}')

pca_var = sklearn.metrics.explained_variance_score(flat_kernel, X_re_orig.T)
print(f'Sklearn PCA: Explained Variance: {pca_var}')

SNMF: Explained Variance: 0.5767240317161563
Manual SVD PCA: Explained Variance: 0.5952852970918956
Sklearn PCA: Explained Variance: 0.5943224234506488
